In [2]:
cd ..

/home/pi/WORK_DIR/notebooks/Ex_Frank/LABS


In [6]:
pwd

'/home/pi/WORK_DIR/notebooks/Ex_Frank/LABS'

# 1.1

In [8]:
from Lab3.lab3_ex1 import WindowGenerator
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument('--model', type=str, help='model name among mlp cnn and lstm')
parser.add_argument('--labels', type=int, default=0,
                    help='0 for temp forecasting, 1 for hum forecasting, 2 or more for both')
#parser.add_argument('--saved_model_dir', type=str, default=None)
args = parser.parse_args(['--labels=2'])

In [11]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

zip_path = "Lab3/data/jena_climate_2009_2016.csv.zip"

csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)

column_indices = [2, 5]
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

n = len(data)
train_data = data[0:int(n * 0.7)]
val_data = data[int(n * 0.7):int(n * 0.9)]
test_data = data[int(n * 0.9):]

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

input_width = 6
LABEL_OPTIONS = args.labels

generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

# Let's save datasets on disk
tf.data.experimental.save(train_ds, './Lab4/th_train')
tf.data.experimental.save(val_ds, './Lab4/th_val')
tf.data.experimental.save(test_ds, './Lab4/th_test')

To load a previously saved dataset, you need to specify element_spec, a type signature of the elements of the saved dataset, which can be obtained via tf.data.Dataset.element_spec. This requirement exists so that shape inference of the loaded dataset does not need to perform I/O.



In [12]:
tensor_specs = (tf.TensorSpec([None, 6, 2], dtype=tf.float32),
                tf.TensorSpec([None, 2]))
#train_ds = tf.data.experimental.load('./Lab4/th_train', tensor_specs)
#val_ds = tf.data.experimental.load('./Lab4/th_val', tensor_specs)
#test_ds = tf.data.experimental.load('./Lab4/th_test', tensor_specs)

# 1.2

In [9]:
interpreter = tf.lite.Interpreter(model_path="Lab4/models/model_{}.tflite".format("cnn_3"))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
                                  
print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

Number of inputs: 1
Number of outputs: 1
Input name: serving_default_conv1d_1_input:0
Input shape: [1 6 2]
Output shape: [1 2]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [14]:
test_ds = tf.data.experimental.load('./Lab4/th_test', tensor_specs)
test_data = test_ds.map(lambda a, b: a)
test_labels = test_ds.map(lambda a, b: b)

In [79]:
test_data = test_data.unbatch().batch(1)
test_labels = test_labels.unbatch().batch(1)

Let's generate np arrays

In [81]:
# We first iterate over batches
for i, (window, labels) in enumerate(zip(test_data.as_numpy_iterator(), test_labels.as_numpy_iterator())):
    # Secondly, we iterate over windows
    interpreter.set_tensor(input_details[0]['index'], window)
    interpreter.invoke()
    predicted = interpreter.get_tensor(output_details[0]['index'])

    if i==0:
        preds = np.array(predicted, dtype=np.float32)
        true_labels = np.array(labels, dtype=np.float32)
    else:
        preds = np.append(preds, predicted, axis=0)
        true_labels = np.append(true_labels, labels, axis=0)

TRY: np.array(list(dataset.as_numpy_iterator())) 

In [84]:
def calculateMAE(actuals, preds):
    """
    Calculate MAE between 2 np vectors.
    np: numpy

    """
    return np.mean(np.absolute(actuals - preds))

In [85]:
print("Temperature MAE: ", calculateMAE(true_labels[:, 0], preds[:, 0]))
print("Humidity MAE: ", calculateMAE(true_labels[:, 1], preds[:, 1]))

Temperature MAE:  0.25623897
Humidity MAE:  0.7927708
